In [8]:
import arrow
from arrow import Arrow
from github import Github
from typing import Type
import pandas as pd
import numpy
import time
import os

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [9]:
g = Github(os.environ['GITHUB_API_KEY'])

# these filters are used to filter by issues or PRs where the author is a member of Regen's team
# or where the PR was reviewed by a regen team member
team = ['aaronc', 'amaurym', 'clevinson', 'blushi', 'anilcse', 'atheeshp', 'aleem1314', \
        'robert-zaremba', 'cyberbono3', 'likhita-809', 'technicallyty', 'ryanchrypto', 'ruhatch', 'frumioj']
team_authored = " ".join(['author:' + i for i in team])

team_reviewed = " ".join(['reviewed-by:' + i for i in team])

In [3]:
team

['aaronc',
 'amaurym',
 'clevinson',
 'blushi',
 'anilcse',
 'atheeshp',
 'aleem1314',
 'robert-zaremba',
 'cyberbono3',
 'likhita-809',
 'technicallyty',
 'ryanchrypto',
 'ruhatch',
 'frumioj']

In [4]:
FMT = 'YYYY-MM-DD'
REPO = 'cosmos/cosmos-sdk'


def end_of_month(date: Arrow) -> Arrow:
    return date.replace(day=1).shift(months=+1, days=-1)

def in_this_month(date: Arrow) -> str:
    return date.replace(day=1).format(FMT) + '..' + end_of_month(date).format(FMT)

def before_this_month(date: Arrow) -> str:
    return '<' + date.replace(day=1).format(FMT)

def after_this_month(date: Arrow) -> str:
    return '>' + end_of_month(date).format(FMT)

def existing_tasks_open(date: Arrow, filt: str="") -> int:
    created_filter = before_this_month(date)
    closed_filter = after_this_month(date)
    query = f'repo:{REPO} {filt} created:{created_filter}'
    closed_later = g.search_issues(query + f' closed:{closed_filter}')
    still_open = g.search_issues(query + ' is:open')
    return closed_later.totalCount + still_open.totalCount

def existing_tasks_closed(date: Arrow, filt: str="") -> int:
    created_filter = before_this_month(date)
    closed_filter = in_this_month(date)
    res = g.search_issues(f'repo:{REPO} {filt} created:{created_filter} closed:{closed_filter}')
    return res.totalCount

def existing_tasks_merged(date: Arrow, filt: str="") -> int:
    created_filter = before_this_month(date)
    closed_filter = in_this_month(date)
    res = g.search_issues(f'repo:{REPO} {filt} created:{created_filter} merged:{closed_filter}')
    return res.totalCount

def new_tasks_open(date: Arrow, filt: str="") -> int:
    created_filter = in_this_month(date)
    closed_filter = after_this_month(date)
    query = f'repo:{REPO} {filt} created:{created_filter}'
    closed_later = g.search_issues(query + f' closed:{closed_filter}')
    still_open = g.search_issues(query + ' is:open')
    return closed_later.totalCount + still_open.totalCount

def new_tasks_closed(date: Arrow, filt: str="") -> int:
    created_filter = in_this_month(date)
    closed_filter = in_this_month(date)
    query = f'repo:{REPO} {filt} created:{created_filter} closed:{closed_filter}'
    res = g.search_issues(query)
    return res.totalCount

def new_tasks_merged(date: Arrow, filt: str="") -> int:
    created_filter = in_this_month(date)
    closed_filter = in_this_month(date)
    query = f'repo:{REPO} {filt} created:{created_filter} merged:{closed_filter}'
    res = g.search_issues(query)
    return res.totalCount

def get_issues(date: Arrow, filt: str=""):
    filt = "is:issue " + filt
    data = [existing_tasks_open(date, filt),
         existing_tasks_closed(date, filt),
         new_tasks_open(date, filt),
         new_tasks_closed(date, filt)
    ]
    
    cols = ['Existing Issues (still open)',
        'Existing Issues Closed',
        'New Issues (still open)',
        'New Issues Closed',
    ]
    result = dict(zip(cols, data))
    
    return pd.DataFrame(result, index=[date.date()])

def get_team_tasks(date: Arrow, typ, filt: str=""):
    if typ == 'pr':
        f_type = 'is:pr'
        f_closed = 'merged'
    elif typ == 'issue':
        f_type = 'is:pr'
        f_closed = 'closed'
    else:
        raise ValueError("`typ` field must be 'pr' or 'issue'")
    closed_filter = in_this_month(date)
    data = [g.search_issues(f'repo:{REPO} {f_type} {team_authored} {filt} {f_closed}:{closed_filter}').totalCount,
        g.search_issues(f'repo:{REPO} {f_type} {team_authored} {team_reviewed} {filt} {f_closed}:{closed_filter}').totalCount,
        g.search_issues(f'repo:{REPO} {f_type} {team_reviewed} {filt} {f_closed}:{closed_filter}').totalCount,
        g.search_issues(f'repo:{REPO} {f_type} {filt} {f_closed}:{closed_filter}').totalCount
    ]
    
    cols = ['Team Authored', 'Team Authored & Reviewed', 'Team Reviewed', 'All']
    
    result = dict(zip(cols, data))
    
    return pd.DataFrame(result, index=[date.date()])
    
def get_prs(date: Arrow, filt: str=""):
    filt = "is:pr " + filt
    data = [existing_tasks_open(date, filt),
            existing_tasks_closed(date, filt),
            existing_tasks_merged(date, filt),  
            new_tasks_open(date, filt),
            new_tasks_closed(date, filt),
            new_tasks_merged(date, filt)
    ]
            
    cols = ['Existing PRs (still open)',
        'Existing PRs Closed',
        'Existing PRs Merged',
        'New PRs (still open)',
        'New PRs Closed',
        'New PRs Merged'
    ]
    
    result = dict(zip(cols, data))
    
    return pd.DataFrame(result, index=[date.date()])

def get_all_prs(date: Arrow, filt: str=""):
    filt = "is:pr " + filt

    closed_filter = in_this_month(date)
    query = f'repo:{REPO} {filt} merged:{closed_filter}'
    prs = g.search_issues(query)

    arr = [["user", "title", "milestone", "closed_at", "closed_by", "number"]]
    for pr in prs:
        res = [pr.user.login, pr.title, pr.milestone, pr.closed_at, pr.closed_by.login, pr.number]
        arr.append(res)
    
    
    return pd.DataFrame(arr[1:], columns=arr[0])

In [5]:
prs_df = pd.DataFrame()
months_to_process = ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07']

In [6]:
for dt in months_to_process:
    time.sleep(20)
    arr = arrow.get(dt)
    prs_df = prs_df.append(get_prs(arr))
    months_to_process = months_to_process[1:]

In [7]:
prs_df

,Existing PRs (still open),Existing PRs Closed,Existing PRs Merged,New PRs (still open),New PRs Closed,New PRs Merged
2021-01-01,29,18,10,27,121,108
2021-02-01,19,37,27,31,124,112
2021-03-01,30,20,16,20,171,149
2021-04-01,21,29,19,23,105,97
2021-05-01,18,26,18,26,102,85
2021-06-01,24,20,16,38,88,77
2021-07-01,23,39,24,27,109,91


In [8]:
issues_df = pd.DataFrame()
months_to_process = ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07']

In [10]:
for dt in months_to_process:
    time.sleep(12)
    arr = arrow.get(dt)
    issues_df = issues_df.append(get_issues(arr))
    months_to_process = months_to_process[1:]

In [11]:
issues_df

,Existing Issues (still open),Existing Issues Closed,New Issues (still open),New Issues Closed
2021-01-01,402,25,50,29
2021-02-01,391,61,52,24
2021-03-01,399,44,69,49
2021-04-01,417,51,38,20
2021-05-01,425,30,26,23
2021-06-01,419,32,27,30
2021-07-01,404,42,54,16


In [12]:
team_prs = pd.DataFrame()
months_to_process = ['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07']

In [13]:
for dt in months_to_process:
    time.sleep(9)
    arr = arrow.get(dt)
    team_prs = team_prs.append(get_team_tasks(arr, 'pr'))
    months_to_process = months_to_process[1:]
team_prs

,Team Authored,Team Authored & Reviewed,Team Reviewed,All
2021-01-01,46,38,70,118
2021-02-01,37,32,89,139
2021-03-01,47,33,93,165
2021-04-01,31,28,80,116
2021-05-01,46,45,82,103
2021-06-01,30,25,80,93
2021-07-01,42,40,101,115
